### https://techblog-history-younghunjo1.tistory.com/372

In [36]:
import numpy as np

# 1. Loss Function
- 손실함수란 "파라미터를 어떤 방향으로 학습시킬지에 대한 가이드라인"
- 신경망 학습 프로세스
    1. 어떤 것을 가지고 학습? 데이터
    2. 무엇을 학습? 가중치, 편향
    3. 어떤 기준으로 학습? 손실 함수 값이 0이 되도록 하습
    4. 0이 되도록 어떻게? 손실 함수의 기울기를 활용해 가중치, 편향의 변화량 얻기
    5. 어떻게 기울기를 계산?
        a. 수치 미분을 통해 계산 (계산이 쉽지만 속도 느림)
        b. 오차 역전파 통해 계산 (계산이 복잡하지만 빠름)
    6. 얻은 변화량 만큼 가중치, 편향 업데이트

In [37]:
# 1. 오차제곱합 (Sum of Squares for Error, SSE)
def sse(y, y_pred):
    loss = np.sum((y_pred - y) ** 2)
    return loss

y = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
y_pred = np.array([0.1, 0.05, 0.6, 0, 0.05, 0.1, 0, 0.1, 0, 0])
loss = sse(y, y_pred)
loss

0.19500000000000006

In [38]:
# 2. 교차 엔트로피 오차 (Cross-Entropy Error, CEE)
def cross_entropy(y, y_pred):
    delta = 1e-7
    print(y * np.log(y_pred + delta))
    loss = -np.sum(y * np.log(y_pred + delta))
    
    return loss
loss = cross_entropy(y, y_pred)
loss

## 현재 데이터
### y : 0과 1로 이루어진 one-hot vector 형태의 데이터
### y_pred : softmax의 확률 예측 값 형태
### 교차 엔트로피 오차는 실제 값 y가 1일 때만 계산이 들어가는 형태이며, 자연로그를 계산하게 된다.
#### -> 오른쪽 수식의 값이 작을수록 교차 엔트로피 오차는 작다는 것을 의미한다.
#### 해당에서 소개한 수식은 데이터 1개에 대한 수식이다.

[-0.         -0.         -0.51082546 -0.         -0.         -0.
 -0.         -0.         -0.         -0.        ]


0.510825457099338

In [39]:
# 3. 교차 엔트로피 오차, 미니배치를 위한
def cross_entropy_ohe(y, y_pred):
    # 데이터가 한개만 들어왔을 경우 2차원의 데이터로 변환
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(1, y_pred.size)
        y = y.reshape(1, y.size)
    # 한 배치당 데이터 갯수
    batch_size = y.shape[0]
    # batch_size를 나누어주는 정규화(normalization) 작업 추가
    loss = -np.sum(y * np.log(y_pred + 1e-7)) / batch_size
    return loss

In [40]:
# 4. 교차 엔트로피 오차, 미니배치를 위한, 이제 레이블 인코딩 된
def cross_entropy(y, y_pred):
    # 데이터가 한개만 들어왔을 경우 2차원의 데이터로 변환
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(1, y_pred.size)
        y = y.reshape(1, y.size)
    # 한 배치당 데이터 갯수
    batch_size = y.shape[0]
    # batch_size를 나누어주는 정규화(normalization) 작업 추가
    loss = -np.sum(np.log(y_pred[np.arange(batch_size), y] + 1e-7)) / batch_size
    return loss

### 손실함수의 의미
- 정확도(Accuracy)와 같은 미분이 하는 역할의 수식은 불연속적인 값을 내뱉는 경향이 강하기 때문에 적절하지 못하다.
- 손실함수는 입력값을 받아 활성함수를 결과값을 기반으로 실행이 되어지는데, 활성함수를 이산적인 값들만 내뱉게 되는 계단함수를 사용하게 되면, 손실함수도 미분하게 적절한 모양으로 나오지 않을 것이 분명하다.

# 2. 수치 미분

In [45]:
# 변화량 값을 계산하기 위해 활용되는 수치미분
# 이제 중심(중앙) 차분으로의 변형 형태 (h를 두번 사용)
def numerical_diff(f, x):
    # 거의 고정값
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

# 3. 편미분
- 수치 미분은 1개의 변수에 대한 미분을 의미했다면, 편미분은 변수가 2개 이상일 때의 미분을 의미한다.

In [46]:
# 1. 2차 함수로 구성된 새로운 함수 정의
def function_2(x: np.array):
    return np.sum(x ** 2)

In [47]:
# 2. 1의 함수를 편미분
# 한 변수씩 미분을 차례대로 수행해준다.
# 이 때, x1값은 상수로 고정한다.
def square_func_temp1(x0):
    y = x0 * x0 + 4.0 ** 2 # x1을 4.0으로 고정한 형태
    return y

# 아래의 수식의 의미
# x1을 상수로 고정시킨 후 x0가 3일 때, h(1e-4)만큼 늘리면 f값은 얼마나 변화했는가를 의미
print(numerical_diff(square_func_temp1, 3.0))

6.00000000000378


In [48]:
# x0을 상수로 고정시키고 x1에 대한 편미분 수행
def square_func_temp2(x1):
    y = 3.0 ** 2 + x1 * x1 # x0을 3.0으로 고정한 형태
    return y

print(numerical_diff(square_func_temp2, 4.0))

7.999999999999119


# 4. 기울기
- 모든 변수의 편미분을 벡터(행렬)로 정리한 것을 바로 기울기(Gradient)라고 한다.
- 즉, 여러 변수들의 변화량들을 모아놓은 것을 이야기 한다.

In [49]:
def function_2(x: np.array):
    return np.sum(x ** 2)

def numerical_gradient(f, x:np.array):
    h = 1e-4
    
    # 기울기를 담아놓은 벡터 행렬 초기화
    gradients = np.zeros_like(x)
    
    # array 원소 하나씩 편미분 수행
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x*h)
        x[idx] = tmp_val + h
        fxh1 = f(x) # f값 계산을 위해서는 모든 x들을 필요로함
        # f(x-h)
        x[idx] = tmp_val - h
        fxh2 = f(x)
        # 미분 공식 수행
        gradients[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = tmp_val # 다른 변수의 편미분을 수행해주기 위해서 -h/+h 했던 원소값을 복원
        return gradients
    
print(numerical_gradient(function_2, np.array([3.0, 4.0])))
print(numerical_gradient(function_2, np.array([8.0, 10.0])))
print(numerical_gradient(function_2, np.array([-5.0, -3.0])))

# 기울기 값의 의미
# 기울기의 변화량 값이 음수이면, 가중치와 편향 값들을 양의 방향으로 변화시켜야 함을 의미하고,
# 변화량 값이 양수라면, 가중치와 편향 값들을 음의 방향으로 변화시켜야 함을 의미한다.
# 변화량들의 집합인 기울기는 일종의 '벡터' 이다. 이것들은 대소비교의 의미가 아닌, '방향' 성을 가리킴을 인지하자

[6. 0.]
[16.  0.]
[-10.   0.]


# 5. 경사하강법 (Gradient Descent)
- 기울기(gradients)로 각 가중치와 편향 값들을 갱신해주는 방법
- 경사하강법이란 기울기 값이라는 방향성을 가이드로 삼아 손실함수 값의 최솟값 또는 최댓값을 찾으려는 것을 이야기 한다.
- 하지만 기울기가 가리키는 방향으로 간다고 해서 무조건 손실함수 값을 최솟값 또는 최댓값으로 간다는 보장은 없다. 이럴 때, 우리가 찾은 값이 극솟값(Local Minimum)인지 최솟값(Global Minimum)인지 모르다고 이야기 한다.
- 하이퍼 파라미터로 학습률(Learning rate)을 받는데, 이는 파라미터 갱신 시 얼마만큼 갱신시켜야 할지를 의미한다.

In [50]:
# 변수가 2개일 때의 경사하강법
def function_2(x: np.array):
    return np.sum(x ** 2)

def gradient_descent(f, init_x: np.array, lr=0.01, step_num=100):
    x = init_x # 100번의 경사하강 수행
    for _ in range(step_num):
        # 1. 손실 함수인 function_2를 기반으로 기울기를 반복 계산
        gradients = numerical_gradient(f, x)
        # 2. x값 업데이트
        x -= lr * gradients
    return x

init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x, lr=0.1, step_num=100))

[-6.10953066e-10  4.00000000e+00]


# 6. Learning Algorithm, Two Layer

In [57]:
# Activation Function: Sigmoid, Softmax
def sigmoid(x):
    return 1 / (1 + np.exp(-x))    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [58]:
# loss function
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [170]:
# gradient function
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
#         print(fxh1)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
#         print(fxh2)
        grad[idx] = (fxh1 - fxh2) / (2*h)
#         print(grad[idx])
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

## 1. Process

### Data

In [181]:
# Import Data 
from datas.mnist import load_mnist

# datas
(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=False)

# history datas
train_loss = []

# Hyper Parameter
epochs = 10
train_size = X_train.shape[0]
batch_size = 100
learning_rate = 0.1

### Model

In [191]:
# Params. Weights, Bias Setting
input_size = 784
hidden_size=50
output_size=10

params = {}
params['W1'] = np.random.randn(input_size, hidden_size)
params['b1'] = np.zeros(hidden_size,)
params['W2'] = np.random.randn(hidden_size, output_size)
params['b2'] = np.zeros(output_size,)

In [192]:
# Batch
batch_idx = np.random.choice(train_size, batch_size)

X_batch = X_train[batch_idx]
y_batch = y_train[batch_idx]

In [193]:
# predict
def predict():
    layer_size = int(len(params.keys()) / 2)

    w1, w2 = [params['W{}'.format(_ + 1)] for _ in range(layer_size)]
    b1, b2 = [params['b{}'.format(_ + 1)] for _ in range(layer_size)]

    # Layer 1
    output = np.matmul(X_batch, w1) + b1
    output = sigmoid(output)
    # Layer 2
    output = np.matmul(output, w2) + b2
    output = softmax(output)

    return output

In [194]:
# 기울기를 위한 손실함수 생성
def loss_func(W):
    y_pred = predict()
    return cross_entropy_error(y_pred, y_batch)

loss_value = loss_func(None)
print("before gradients, loss value : {}".format(loss_value))

before gradients, loss value : 7.7230217886787


In [195]:
# 기울기 계산
gradients = dict()

for key in params.keys():
    gradients[key] = numerical_gradient(loss_func, params[key])
    
# 위에 작성한 손실함수를 변경 시켜준 것을 확인할 수 있다.
# 손실함수에서 사용하는 글로벌 변수들의 연산으로 나오는 y_pred와 y_batch는 고정적인 글로벌 값이 되는데 계속 같은 값의 비교만 진행하면
# 기울기가 나타나지 않지 않나?

# 핵심은 params[key]에 있다. 가중치 혹은 편향을 매개변수로 주는데 이 때의 호출 방식은 call by reference 이다.
# 즉, numeric_gradient에서 다루고 있는 것들은 전역 가중치 혹은 편향이다.
# 하나씩 변경해 가면서 predict()를 해주는데 이 때 변화를 주는 가중치 혹은 편향의 값이 있을 것 이다.
# 이에 따라 변화량을 기록한다. 그 후, 다음 아이템의 연산을 위하여 원본값으로 변경시켜준다.

In [196]:
# 학습율에 따른 기울기 적용
for key in params.keys():
    params[key] -= learning_rate * gradients[key]

In [197]:
# 줄어든다,, 대박,,
loss_value = loss_func(None)
print("after gradients, loss value : {}".format(loss_value))

after gradients, loss value : 7.2064644383203005
